
# Master Thesis Presentation Notebook
## by Théo Stassen

This notebook presents the test protocol implemented for the purpose of the Master Thesis. 
The goal is the comparison of different probabilistic forecasting models, mainly from the deep learning field, 
in the context of the prediction of wind turbines production.

The code uses GluonTS, a toolkit which contains components and tools for building time series models using MXNet

Concerning the installation, to run of the code, you need to be installed 
(see https://gluon-ts.mxnet.io/install.html) :

- mxnet, using pip, in a compatible version with gluon ts (>=1.3.1,< 1.5.*, cpu only or with gpu support)  
- If you want gpu support, you need a compatible Nvidia gpu, and a compatible version (8.0, 9.0, or 9.2) of 
CUDA (check if working properly)
- gluonts, using pip

In [ ]:
# Third party imports
import pandas as pd
import numpy as np
import dataset as dt # Custom file containing class definition of Dataset

# In case of this error : 
# OSError: libcudart.so.9.2: cannot open shared object file: No such file or directory 
# if you are on (ubuntu) linux, do : sudo ldconfig /usr/local/cuda/lib64

## Dataset Import

The project need some entry data. In this version, the input files are in /datasets.
They  are array of values. GluonTS needs, to create datasets, some information .
   In general, the datasets defined by GluonTS are objects that consists of three main members:

- `dataset.train` is an iterable collection of data entries used for training. Each entry corresponds to one time series
- `dataset.test` is an iterable collection of data entries used for inference. The test dataset is an extended version 
of the train dataset that contains a window in the end of each time series that was not seen during training. 
This window has length equal to the prediction length.
- `dataset.metadata` contains metadata of the dataset such as the frequency of the time series, the context (training) length
a prediction length, associated features (if we want to combine the temporal series information with other data), etc.


  All the models will use the data by splitting the entries (in a way that we can precise) into pieces of size 
  learning ("context") + predict length on which they will train and predict.
  
  In this first version, only one array is imported and all the time series corresponds to the same source. 
  Precisely, we import the `6months-minutes.csv` file. The first idea was to use it as a single time serie. The problem 
  was that the models had very long training time, about 120s per epochs (100 epochs recommended). The data was correctly 
  splitted but Mxnet is not optimised for this situation. The solution is to split manually the 6 month into 
  different time series. The natural choice is to split into days. We have so 183 samples/time series of length 1440 time steps
  with a frequency of 1 minute. The values are also divided by 1000 because it was causing loss problems.
  In future version We could import and combine the data of multiple sources as a single training/testing dataset, 
  or use multiple sources for training and another for testing.
  
  The goal is to predict the production in a window of 10 minutes, which means 10 time steps.
  

In [ ]:
df = pd.read_csv("datasets/6months-minutes.csv")
imported_dataset = np.array([df['Active Power'].to_numpy()])
n_sample = 183
imported_dataset = imported_dataset.reshape(n_sample, -1)  # Split by days
imported_dataset = imported_dataset / 1000
prediction_length = 10
context_length = 60 * 1  # One day
freq = "1min"
start = pd.Timestamp("01-04-2019", freq=freq)
dataset = dt.Dataset(custom_dataset=imported_dataset, start=start, freq=freq,
                     prediction_length=prediction_length,
                     learning_length=context_length)

## Quantiles

The project is to do probabilistic forecasting of wind turbines time series but the goal is not really
to minimize the difference (in any ways to consider it) between the predicted values and the observed values.
The goal is to obtain a distribution of probability of the future values which have the more correct "safety" quantile. 
What is forbidden is that the model tells us a understimated value of the quantile ( a too thin values window).
We want a model which tells us that there are 99% (or another security value) chance that the real value of production 
will be in this window and that affirmation must be exact (or overestimating) not underestimating.
This goal must be combine with the goal to have the thinner estimation window. 
We want a balance between the not understemation of the risk and the "precision" of the predict.

The quantile list contains the values of quantiles that are useful for us. We want a 99% precision so we consider the 
[0.005, 0.995] window and we compute also the 90% and 50% precision to have a gradual comparison.

In [ ]:
quantiles = list([0.005, 0.05, 0.25, 0.5, 0.75, 0.95, 0.995])

## Metrics

To observe if the models do well what we ask to do, to observe their quality considering the goals that are fixed, we 
need to choose the appropriate metrics. GluonTS proposes some metrics to evaluate models (MSE, MSIS, etc).

Concerning the correctness of the quantiles, none of the GluonTS metric is satisfying for this purpose. The proposed 
metric, "Coverage" is a simple combination of two of them. For a security limit of k :  

$Coverage = (k - coverage(k)) - (coverage(1-k) - (1-k)) $

With coverage(x) is a GluonTS implemented metric which give the observed probability to have an value below the quantile x.
If the distribution is perfect the coverage must tends to the value x. For the upper bound, if coverage(x) > x, the quantile is overlarge and
if coverage(x) < x the quantile is underlarge. And it is the opposite for the lower bound.
For a 99% precision the security limit is 0.995. 

$Coverage = (0.995 - coverage(0.995)) - (coverage(0.005) - 0.005) $

To observe if the distribution is large, we compute a custom metric which is simply the mean of the difference between 
the quantile(k) and quantile(1-k). For a time serie :

$$Bandwidth = \frac{1}{p}  \sum_{i=0}^{p} quantile_{i}(k) - quantile_{i}(1-k) $$

Where p is the number of time step and $$quantile_{i}$$ the value of the quantile at time step i


In [ ]:
chosen_metric = "Coverage"



## Custom loss

As explained in the previous sections, we want the model to give precise estimation of the quantiles.
The different models presented in the following sections will give more or less good results but we can interrogate how 
the model consider that it is good and if we want to modify it.
In most of the models, the loss function is the inverse of the log-density of the output distribution.
If $ \mu $ and $ \sigma $ are the parameters of the distribution output (in the case of a gaussian) by the model : 

$$ base\_loss(x) = - ln \varphi_{\mu,\sigma^2}(x)  $$

This loss is the natural way to formulate that we want that the distribution corresponds to the observed x.
But with this loss, the variation of the loss is inversely proportinal to the deviation between x and the max
of the distribution. Between to big deviation, for example if x at the position of the quantile(0.9) and if x is at 
quantile(0.99), the loss is near.
We need another loss with the particular goal to discourage the values of x bigger than quantile(0.99) and lower than 
quantile(0.01). 
The proposed loss is the following :

$$ alt\_loss(x) = e^{x - quantile(0.995)} + e^{quantile(0.005)-x}$$

The loss increase exponentially if x is outside the security window, is lower and not exponential in the window.

In the pre-implemented GluonTs models the only usable loss is $base\_loss$ but if we create custom models 
it is possible to modify it. 
At this time, three custom models (one original, "Simple", and the copy of the pre-implemented "SimpleFeedForward" and 
"CanonicalRNN" are implemented. And they uses this loss :

$$ custom\_loss = base\_loss + \alpha * alt\_loss $$

With $\alpha$ a parameter that we will make vary when comparing all the models together.
Depending of it value the results will be very different.

In [ ]:
alphas = [0.7, 0.8, 0.9]
alpha = 0.9

## Distribution

GluonTs proposes different type of output distribution. In most of the models the output is a vector of values which are
transformed to a vector of distribution parameters and put in a distribution object.
We can precise what kind of distribution.
Three of them are working for now. Gaussian, Laplace and PiecewiseLinear.
Also implemented there are also Student, but it is rejected because we cannot obtain directly the quantiles of the distribution, 
that we need for the loss, and Uniform but it make the loss outbound.
And there are others, more complicated, like the multiple kernel gaussian.

In [ ]:
# distributions = ["Gaussian", "Laplace", "PiecewiseLinear", ]
# No quantiles in Student, Uniform has a problem with loss

distributions = ["Gaussian"]
distribution = "Gaussian"

# Different models

All the deep learning models are presented here. In each section we compare the effect of the main parameters on the
coverage and bandwidth.

In [ ]:
from comparison import compare_all_models, compare_simple, compare_simple_feed_forward, compare_canonicalrnn, \
    compare_deepar, compare_deepfactor, compare_mqcnn, compare_mqrnn

## Example of resulting plots

We can see what some of the models give as forecast results.

In [ ]:
epochs = 100
compare_all_models(dataset,"Gaussian",0.9,["cSimple", "cSimpleFeedForward","cCanonicalRNN"],chosen_metric,epochs,True)


## Global comparison

Here is the comparison of all models, for different alphas and distribution output type
 with all the presented in addition of two models not using deep learning
(ETS and seasonal naive).

In [ ]:

models = ["cCanonicalRNN"]
compare_all_models(dataset, distributions, alphas, models, chosen_metric, epochs,False)
 

## Simple

A model which uses a simple 2 fully connected layers neural network.
The tunable parameter is the number of cells in layers

In [ ]:
num_cells = [10, 50, 100, 200, 300]
compare_simple(dataset, distribution, alpha, chosen_metric, epochs, num_cells)

## FeedForward

A simple MLP model.
The tunable parameter is the number of hidden nodes in each layer.


In [ ]:
num_hidden_dimensions = [[10], [40], [40, 40], [40, 40, 40]]
compare_simple_feed_forward(dataset, distribution, alpha, chosen_metric, epochs, num_hidden_dimensions)

## Recurrent Neural Network

A model which uses a recurrent neural network.
The tunable parameters are the number of layers and number of cells of the network

In [ ]:
# Other parameters : num_cells and embedding dimensions
num_layers = [1, 2, 5, 10]
num_cells = [20,50,100]
compare_canonicalrnn(dataset, distribution, alpha, chosen_metric, epochs, "n_layers", num_layers)
compare_canonicalrnn(dataset, distribution, alpha, chosen_metric, epochs, "n_cells", num_cells)

## Deep Ar

Implementation of DeepAr estimator, a RNN based model, close to the one described in paper 
["DeepAR: Probabilistic Forecasting with Autoregressive Recurrent Networks" ](https://arxiv.org/abs/1704.04110)
The tunable parameters are the same that for CanonicalRNN

In [ ]:
num_layers = [1, 2, 5, 10]
num_cells = [20,40,50,100]
compare_deepar(dataset,distribution,alpha,chosen_metric,epochs,"n_layers",num_layers)
compare_deepar(dataset,distribution,alpha,chosen_metric,epochs,"n_cells",num_cells)

## Deep Factors

Implementation of the 2019 ICML paper [“Deep Factors for Forecasting”](https://arxiv.org/abs/1905.12417)
It uses a global RNN model to learn patterns across multiple related time series and an arbitrary local model to model the time series on a per time series basis. 
In the current implementation, the local model is a RNN (DF-RNN).
The tunable parameters are the number of units per hidden layers and the number of layers in the global model
, the number of global factors. We could also add the number of units and layers in the local model.

In [ ]:
num_hidden_global = [20,50,100]
num_layers_global = [1, 2, 5]
num_factors = [5, 10, 20]
compare_deepfactor(dataset,distribution,alpha,chosen_metric,epochs,"n_hidden_global",num_hidden_global)
compare_deepfactor(dataset,distribution,alpha,chosen_metric,epochs,"n_layers_global",num_layers_global)
compare_deepfactor(dataset,distribution,alpha,chosen_metric,epochs,"n_factors",num_factors)

## Gaussian Process

Model using Gaussian Processes (GP).
Each time series has a GP.
There are no parameters easy to tune

## NPTS

Implementation of the Non-Parametric Time Series Forecaster, 
which falls into the class of simple forecasters that use one of the past observed targets as the forecast for 
the current time step. It randomly samples a past time index as the prediction for the time step T (it is 
an auto regressive model which predict time steps one by one). 
There are no parameters to tune, but some variant are possible.


## MQCNN

Discriminative Sequence to Sequence model made using the SeqtoSeq framework of GluonTS to reproduce the model of the 
paper [A Multi-Horizon Quantile Recurrent Forecaster](https://arxiv.org/abs/1711.11053).
Sequence to sequence models are composed of two parts. The encoder network, that reads in a certain context of the training range of the time series 
and encodes information about  the  sequence  in  a  latent  state.
And the decoder network, which generates the forecast by combining the latent information with the features in the prediction range
In MQCNN the encoder is a Convolutionnal Neural Network and the decoder an mlp model. The output
is not technically a  distribution but the quantiles itself, that are predicted values obtain by optimising
the corresponding quantile loss (In general in a deterministic world if you train a model with as loss the quantile
loss [0.9], you will obtain the line representing the quantile [0.9])
The tunable parameter is the  dimension of the mlp decoder.

In [ ]:
mlp_final_dim = [10,20,30]
compare_mqcnn(dataset, distribution, alpha, chosen_metric, epochs, mlp_final_dim)

## MQRNN

Same as MQCNN but with a Recurrent Neural Network encoder.

In [ ]:
mlp_final_dim = [10,20,30]
compare_mqrnn(dataset, distribution, alpha, chosen_metric, epochs, mlp_final_dim)

# Other plots implemented

## metrics by time series

We can obtain the item metrics, i.e. the metrics for each time series instead of the global metrics for all time
series. These metrics can be plot as histogramm of metric values, to see the repartition of values in the 183 time
series. This works not for all metrics, for coverage the results are not precise. An example is the MSE for the 
three first models. 

In [ ]:
from plots import plot_distr_params, hist_plot_item_metrics
compare_all_models(dataset, distributions, alphas, models, "MSE", epochs, False)
hist_plot_item_metrics("MSE", ["cSimple", "cSimpleFeedForward","cCanonicalRNN"])  
# Not precise for Coverage (too much around)

## distribution information

In the custom models we save the exact parameters of output distribution. We can plot the values here. 

In [ ]:
plot_distr_params(["cSimple", "cSimpleFeedForward","cCanonicalRNN"], alphas, distributions) 
